<div style="text-align:right;color:blue">version id: __VERSION_ID__</div>

# Storing numbers on a computer
In one of the lectures in semester 1 we saw that each variable in Python has a *type*. For example, a number could be stored as an integer (type `int`) or as a real number (type `float`). In this notebook we will have a closer look at how variables are stored in computer memory. Since they are so important in mathematics, we will concentrate on integers and real numbers. While it is good to be aware of the details, the most important thing that you should remember is that the storage format can have an impact on the correctness and accuracy of computations. Common issues are numerical overflow or rounding errors and it is important to be aware of these when interpreting the results of numerical calculations in Python.

<hr style="height: 2px">

### What you will learn
In this notebook we will cover the following topics:

* Storing data on a computer: bits and bytes
* Representation of integer numbers
* Numerical overflow
* Real numbers and the floating point format
* Common issues with floating point numbers
* Guidelines for working with numbers in Python

<hr style="height: 2px">

*&#169; Eike Mueller, University of Bath 2019-2025. These lecture notes are copyright of Eike Mueller, University of Bath. They are provided exclusively for educational purposes at the University and are to be downloaded or copied for your private study only. Further distribution, e.g. by upload to external repositories, is prohibited.*

## Storing data on a computer
Internally, computers can actually only store two different values. A particular memory location, which is called a *bit*, can be either 0 (off) or 1 (on). Individual bits are combined into larger units, and a collection of 8 bits is also known as a *byte*. Since there are $2^8=256$ different combinations of eight 0's and 1's, a byte can represent $256$ different states. For example we could use a single byte to represent an integer number in the range between $0$ and $255$ inclusive.
Several bytes can be combined to store larger numbers. For example, 4 bytes (=32 bits) can be used to represent an integer number in the range $0,\dots, 2^{32}-1 \approx 4\cdot 10^9$. The corresponding numpy datatype is known as `uint32`, which is short for ''*unsigned integer represented by 32 bits*''.

![Bits and bytes used to represent an `uint32` integer](./figures/BitsAndBytes.svg)

## Representation of integer numbers
Since many of the computations we will do in mathematics use the numpy library, we will focus on the numpy representation of numbers here. To illustrate this, let us create a small $2\times 2$ matrix and tell numpy to store the numbers in the `uint32` format, which uses 32 bits (=4 bytes) to store an unsigned number. This can be done by passing the keyword `dtype=np.uint32` to the `np.array()` function which creates the matrix.

In [ ]:
import numpy as np
A = np.array([[75,34],[21,13]],dtype=np.uint32)

If we do not specify a datatype, numpy will make its own choice. We can check the data format that numpy uses by looking at the `.dtype` property of the object it created. Here is an example:

In [ ]:
A = np.array([[75,34],[21,13]],dtype=np.uint32)
B = np.array([[75,34],[21,13]])
print ('type of A:', A.dtype)
print ('type of B:', B.dtype)


`int64` is another datatype, which uses 64 bits (=8 bytes) to represent a *signed* number. This demonstrates that by default numpy uses 8 byte = 64 bit to store the entries of the matrix $B$ as signed integers.

Note that explicity telling Python that the entries of `A` should be of type `uint32` implies that we can not set the entries of the matrix to negative values. If we try to do this we get unexpected results, as the following code demonstrates:

In [ ]:
A[0,0] = -25
print(A)

### Unsigned integers
So now let us have a look at how exactly integer values are represented as combinations of 0's and 1's. To keep things simple, we will first look at the `uint8` datatype, which - as you will recall from above - uses 1 byte = 8 bits to represent unsigned (=non-negative) integer numbers. We use the subscript ${}_2$ to denote a sequence of bits, for example $01101001_2$. This is also known as the *binary representation*. In the `uint8` format the numbers in the range $0,1,2,\dots,255$ are stored as follows:

$$
\begin{aligned}
00000000_2 &= 0\\
00000001_2 &= 1\\
00000010_2 &= 2\\
&\dots\\
01001011_2 &= 1\cdot2^0+1\cdot2^1+1\cdot 2^3+1\cdot2^6=75\\
&\dots\\
11111111_2 &= 255
\end{aligned}
$$

This can be written down more mathematically as follows. Let $b_j\in\{0,1\}$ be the value of the $j$-th bit in the byte (counting from the *right*). Then the eight bits $b_0,b_1,\dots,b_7$ represent an unsigned integer number $w$ as follows:

$$
(b_7b_6b_5b_4b_3b_2b_1b_0)_2 = w = \sum_{j=0}^{7} b_j \cdot 2^j, \quad b_j \in \{0,1\}
$$

The leftmost bit ($=b_7$) is known as the *most significant bit (MSB)* since it is the coefficient of the highest power of $2$ in the sum. The rightmost bit ($=b_0$) is called the *least significant bit (LSB)*.

We can obtain the binary representation of a number in Python by using the built-in function `bin()`. For example, to print out the binary representations of the numbers 2 and 75, we can use

In [ ]:
print(bin(2))
print(bin(75))

As we can see from this example, `bin()` returns the binary representation as a string starting with `0b` followed by a sequence of bits, dropping any leftmost zero bits.

The following table gives an overview over the *unsigned* integer types provided by numpy.

| type | number of bits | number of bytes | range |
| :---: | :---: | :---: | :---: |
| `np.uint8` | 8 | 1 | $$0,\dots,2^8-1=255$$ |
| `np.uint16` | 16 | 2 | $$0,\dots,2^{16}-1=65535$$ |
| `np.uint32` | 32 | 4 | $$0,\dots,2^{32}-1\approx 4\cdot 10^{9}$$ |
| `np.uint64` | 64 | 8 | $$0,\dots,2^{64}-1\approx 1.6\cdot 10^{19}$$ |

The binary representation of those types is similar to the one for `np.uint8`. A number of type `np.uint32` is represented as

$$
(b_{31}b_{30}b_{29}\dots b_3b_2b_1b_0)_2 = \sum_{j=0}^{31} b_j\cdot 2^j,\quad b_j\in\{0,1\}
$$

For example (adding spaces to separate the four different bytes)

$$
00001110\;\;11111110\;\;00100001\;\;00000001_2 = 251535617
$$

### Signed integers
So far we have only considered unsigned integer types, which can be used to represent non-negative numbers. To store signed integers we can use the so-called *Two's complement* format. This uses the most significant bit (MSB) $b_{N-1}$ of an $N$ bit binary representation to encode the sign of the number. The binary representation represents a positive number if $b_{N-1}=0$ and it represents a negative number if $b_{N-1}=1$. To make it clear that the binary representation encodes a signed integer we use a minus sign as a superscript in addition to the subscript 2. A given sequence of bits $b_0,b_1,\dots,b_{N-1}$ represents a signed integer as follows:

$$
(b_{N-1}b_{N-2}b_{N-3}\dots b_2b_1b_0)_2^- = -b_{N-1}2^{N-1} + \sum_{j=0}^{N-2} b_j 2^j
$$

For example we have $11100111_2^- = -2^7 + \left(2^0 + 2^1 +2^2 + 2^5 + 2^6\right) = -25$. The formula above can be written in a slightly different form, depending on the value of the MSB:

* $b_{N-1}=0$ (positive integers)
$$
(\boldsymbol{0}b_{N-2}b_{N-3}\dots b_2b_1b_0)_2^- = \sum_{j=0}^{N-2} b_j2^{j} = (b_{N-2}b_{N-3}\dots b_2b_1b_0)_2
$$

* $b_{N-1}=1$ (negative integers)
$$
\begin{aligned}
(\boldsymbol{1}b_{N-2}b_{N-3}\dots b_2b_1b_0)_2^- &=
  -1-\sum_{j=0}^{N-2}\overline{b}_j2^j\qquad\text{with $\overline{b}_j=1-b_j$}\\
  &= -1 - (\overline{b}_{N-2}\overline{b}_{N-3}\dots \overline{b}_2\overline{b}_1\overline{b}_0)_2
  \end{aligned}
$$

Putting a bar above $b_j$ flips the $j$-th bit, i.e. $\overline{1}=0$ and $\overline{0}=1$. A proof of the equation for $b_{N-1}=1$ can be found at the very end of this notebook.

Let us look at a few more examples for an 8 bit storage format which illustrate this:

$$
\begin{aligned}
00000000_2^- &= 0 & 11111111_2^- = -1 -0000000_2  &= -1\\
00000001_2^- &= 1 & 11111110_2^- = -1 -0000001_2  &= -2\\
00000010_2^- &= 2 & 11111101_2^- = -1 -0000010_2  &= -3\\
00000011_2^- &= 3 & 11111100_2^- = -1 -0000011_2  &= -4\\
& \dots & &\dots\\
01111111_2^- &= 127 & 10000000_2^- = -1 -1111111_2 &= -128
\end{aligned}
$$

In general, an $N$ bit signed integer data type can store numbers in the range $-2^{N-1},\dots,2^{N-1}-1$. The properties of numpy signed integer datatypes are summarised in the following table.

| type | number of bits | number of bytes | range |
| :---: | :---: | :---: | :---: |
| `np.int8` | 8 | 1 | $$-2^7,\dots,2^7-1 = -128,\dots,+127$$ |
| `np.int16` | 16 | 2 | $$-2^{15},\dots,2^{15}-1=-32768,\dots,+32767$$ |
| `np.int32` | 32 | 4 | $$-2^{31},\dots,2^{31}-1\approx -2\cdot 10^{9},\dots,+2\cdot 10^{9}$$ |
| `np.int64` | 64 | 8 | $$-2^{63},\dots,2^{63}-1\approx -8\cdot 10^{18},\dots,+8\cdot 10^{18}$$ |

Again, we can gain insights into the binary representation of a number with the `bin()` function. Consider, for example, the number $-25$. To see that Python really uses two's complement, we can try to store this number as an unsigned integer and inspect its representation:

In [ ]:
a_minus25 = np.uint8(-25)
bin(a_minus25)

This agrees with the discussion above. Note, however, that `a_minus25` is treated as a *signed* integer by Python, so its actual value is

$$
2^7 + 2^6 + 2^5 + 2^2 + 2^1 + 2^0 = 231
$$

and this is confirmed by printing it out:

In [ ]:
print(a_minus25)

### Overflow errors
As the two tables above show, the set of integers that can be represented by the numpy datatypes is large but finite. This can have practical implications. Assume, for example that we type to add two matrices $A$ and $B$ with very large, positive entries:

$$
A = \begin{pmatrix}
4611686030842740266 & 21515 \\
352163116236 & 71521362
\end{pmatrix}\qquad
B = \begin{pmatrix}
9223372036854524557 & 5121151 \\
33272516 & 125151
\end{pmatrix}
$$

Obviously, since all entries of $A$ and $B$ are positive, their sum $A+B$ should also have positive entries. Let's try this out:

In [ ]:
A = np.array([[4611686030842740266,21515],[352163116236,71521362]])
B = np.array([[9223372036854524557,5121151],[33272516,125151]])
print(A + B)

The entry in the upper left corner is negative! This can not be correct. The problem is that the sum of the upper left entries of $A$ and $B$ is a number which can not be represented by the `int64` datatype (recall that this is the default datatype chosen by Python for integer-valued matrices) since $4611686030842740266+352163116236=13835058067697264823 > 2^{63}-1$. This is an example of an **overflow error**, since Python is not able to store the result with the available datatype. Worse, Python simply returns an obviously incorrect result without alerting us to the fact that something is going wrong here. You can probably imagine that this can lead to very nasty errors in your code.

In fact, this is not just an academic problem but can have real-life consequences: the [explosion of the Ariane 5 rocket in 1996](http://www-users.math.umn.edu/~arnold/disasters/ariane.html) was caused by a 16bit overflow error.

The key lesson is that whenever you do calculations in Python you should always be aware of potential overflows and interpret the results of your calculation with caution.

## Native Python integers
You might object that Python actually lets you add very large numbers without any problems. The following code adds the two large numbers $a=4611686030842740266$ and $b=9223372036854524557$ without any problem and gives the correct result:

In [ ]:
a = 4611686030842740266
b = 9223372036854524557
a+b

We can even create extremely large integer numbers such as $3^{1000}$. Try it out:

In [ ]:
3**1000

This is because Python stores integers that are not entries of numpy matrices in a different format, which is not limited by the constraints discussed above. However, there are two problems with this:

1. Manipulating numbers in the native Python integer datatype is slower that working with integers in the numpy formats
2. We can not (easily) use the native Python integers inside matrices which are needed in many mathematical applications

## Real numbers
Next, let us consider real numbers. While it might be possible to store rational numbers such as $\frac{2}{7}$, $-\frac{8}{13}$ or $\frac{56181581}{2154152778}$ as pairs of integers, particular challenges arise for irrational numbers such as

$$
\begin{aligned}
\pi &= 3.141592653589793238462643383279502884197169399375105\dots\\
\sqrt{2} &= 1.414213562373095048801688724209698078569671875376948\dots
\end{aligned}
$$

which have an infinite number of digits. Real numbers can be very large such as

$$
678285322700000000000.0
$$

or very small (i.e. close to zero)

$$
-0.0000000000000000000000000000000000000132451
$$

Clearly it is not possible to store an infinite number of digits (which would be necessary to represent $\pi$ or $\sqrt{2}$) on a computer with finite memory, so we have to make some compromise here. The widely used solution is to represent real numbers in so-called *floating point format*.

### Floating point format
For this we first rewrite the numbers in *scientific notation*

$$
\begin{aligned}
\color{blue}{6782853227}00000000000.0 &= \color{green}{+}\color{blue}{6.782853227} \cdot 10^{\color{red}{+20}}\\
-0.0000000000000000000000000000000000000\color{blue}{132451} &= \color{green}{-}\color{blue}{1.32451}\cdot 10^{\color{red}{-38}}
\end{aligned}
$$

Note how the decimal point moves or ''floats'' in those expressions. The absolute value of the number in front of the decimal point on the right hand side of the identities is always in the range $1,\dots,9$.

If a number has too many digits, such as $\pi$ or $\sqrt{2}$, those will get truncated. Consider for example

$$
\begin{aligned}
0.00\color{blue}{6513241}3526788\dots \mapsto \color{green}{+}\color{blue}{6.513241}\cdot 10^{\color{red}{-3}}\\
\color{blue}{-1474785}7514127125.67657\dots \mapsto \color{green}{-}\color{blue}{1.474785}\cdot 10^{\color{red}{+16}}
\end{aligned}
$$

From this we conclude that in most case real numbers are only stored **approximately** on a computer. This implies that special care has to be taken when working with real number, we will look at some examples and typical issues at the end of this notebook.

In general, a floating point number can be written in the form

$$
\color{green}{\pm} \color{blue}{\text{mantissa}}\cdot 10^{\color{red}{\text{exponent}}}
$$

In other words, a floating point number can be represented by three components:

1. the $\color{green}{\text{sign}} ($+$ or $-$)$
2. the $\color{blue}{\text{mantissa}}$
3. the $\color{red}{\text{exponent}}$

Python will separate mantissa and exponent by the letter `e` or `E`. Consider for example:

In [ ]:
large_number = 1.512E-7
small_number = -0.0000000000000000000000000000000000000132451
print(large_number)
print(small_number)

### Storing floating point numbers
Python represents a floating point number $\omega$ by the three numbers $\color{green}{S}$ (sign), $\color{blue}{M}$ (mantissa) and $\color{red}{E}$ exponent as follows:

$$
\omega = (-1)^{\color{green}{S}} \color{blue}{M}\cdot 2^{\color{red}{E}}
$$

Note that $\omega$ is positive if $S=0$ and negative if $S=1$.

The [IEEE 754 standard](https://ieeexplore.ieee.org/document/8766229) states that for a double precision floating point number $\color{green}{S}$,$\color{blue}{M}$ and $\color{red}{E}$ are encoded as follows in 64 bits:

![IEEE 754 representation of double precision floating point numbers in 64 bits](./figures/FloatingPointBits.svg)

* The $\color{green}{\text{sign}}$ is encoded in the most significant (leftmost) bit 
$$\color{green}{S=b_{63}}$$
* The $\color{red}{\text{exponent}}$ is encoded as an 11 bit integer shifted by $2^{10}-1=1023$:
$$\color{red}{E = (b_{62}b_{61}\dots b_{53}b_{52})_2-1023}$$
* The $\color{blue}{mantissa}$ is stored as a 53 bit number (which has a binary representationthat is implicitly assumed to start with 1) as
$$\color{blue}{M=(1.b_{51}b_{50}\dots b_2b_1b_0)_2} = 1 + \sum_{j=0}^{51} b_j 2^{j-52}$$

Some values are reserved for special cases such as

* $E=11111111111_2$, $M=0$: $\pm\infty$
* $E=11111111111_2$, $M\ne0$: NaN (''Not a Number'')

The details are not terribly important. What you should remember is:

* the floating point format can only represent numbers of a magnitude in a certain range. For 64 bit double precision numbers this range is approximately $2.2\cdot 10^{-308},\dots,1.8\cdot 10^{+308}$
* in this range, the numbers are truncated

Between the real numbers that can be represented as floating point numbers there are gaps of increasing size. This is not surprising, since it is obviously impossible to represent all real numbers in the range $0,\dots,1.8\cdot 10^{+308}$ with 64 bits, which can only take on $2^{64}\approx1.8\cdot 10^{19}$ different values. The following figure illustrates this: all real numbers which can be represented as floating point numbers are marked by vertical lines.

![Gaps between floating point numbers](./figures/FloatingPointGaps.svg)

The size of the gaps increases roughly in proportion to the absolute size of the numbers. Because of those gaps, the results of calculations are rounded. For example, if $a_{\text{float}}$, $b_{\text{float}}$ and $c_{\text{float}}$ are floating point numbers, then addition is not necessarily associative, i.e. in general

$$
a_{\text{float}} + (b_{\text{float}}+c_{\text{float}}) \ne (a_{\text{float}} + b_{\text{float}}) + c_{\text{float}}\qquad(\dagger)
$$

This problem is particularly serious if for operations that involve numbers of vastly differing size, such as $a_{\text{float}}=2\cdot 10^{+12}$, $b_{\text{float}}=-2\cdot 10^{+12}$ and $c_{\text{float}} = 4\cdot 10^{-2}$. The following code demonstrates this by evaluating both sides of Eq. ($\dagger$):

In [ ]:
a_float = 2.E+12   # very large number
b_float = -2.E+12  # another very large number
c_float = 4.E-2    # very small number
summation_1 = (a_float + b_float) + c_float
summation_2 = a_float + (b_float + c_float)
print ('(a + b) + c = ',summation_1) # Add a to b first, then add c
print ('a + (b + c) = ',summation_2) # Add b to c first, then add this to a
# Relative difference (in %)
rel_diff = 100.*(summation_2 - summation_1)/summation_1
print ('relative difference = ',rel_diff,' %')

In this example the results differ by $0.1\%$, which is not insignificant. 

The size of the gap around $1$ is quantified by a number known as *machine epsilon* $\varepsilon_{\text{mach}}$. There are different definitions of this quantity, and the one we will use here is the following:

**Definition:**

$\varepsilon_{\text{mach}}$ is the largest number such that for $x_{\text{tiny}}$ with $|x_{\text{tiny}}|<\varepsilon_{\text{mach}}$ Python can not distiguish the number $1+x_{\text{tiny}}$ from $1$ in floating point arithmetic.

With this definition, the gap of floating point numbers is $2\varepsilon_{\text{mach}}$ as shown in the above figure. In the literature the same quantity is sometimes also called *'unit in the last place'* or *'unit of least precision'* (ULP).

For double precision numbers the machine epsilon is $\varepsilon_{\text{mach}}= 2^{-53}\approx 1.1\cdot 10^{-16}$.

Let's try this out:

In [ ]:
x_tiny = 1.E-16 # choose a number smaller than the machine epsilon
1+x_tiny == 1   # is 1+x identical to 1?

Note that this does *not* imply that $x_{\text{tiny}}$ actually is zero:

In [ ]:
x_tiny = 1.E-16
x_tiny == 0     # is x identical to 0?

Try changing $x_\text{tiny}$ to $1.12\cdot 10^{-16}$ and convince yourself that $x_{\text{tiny}}+1\ne 1$ (in floating point arithmetic) in that case.

Here are a few more samples that show that we need to be careful when working with floating point numbers:

In [ ]:
print(1 - 3*(4/3-1))        # this should be 0
print(1.E16 + (-1.E16 + 1)) # this should be 1
print((1.E16+1)**0.5-1.E8)  # this should be approximately 0.5E-8

Those types of errors are known as *rounding errors*. Again, Python just silently returns an incorrect result. Rounding errors tend to accumulate, i.e. build up over a calculation. Although they might look less severe the overflow errors, they can have real consequences, as in the catastrophic [patriot missile failure in the 1990s](http://www-users.math.umn.edu/~arnold//disasters/patriot.html).

As a rule of thumb, if we carry out calculations with numbers of comparable size the *relative* error due to rounding of floating point numbers is of the order of the machine epsilon. This means the following: assume that the true result of some calculation is $Q_{\text{true}}$, but that Python will return a rounded result $Q\approx Q_{\text{true}}$ The relative error defined by

$$
\Delta = \left|\frac{Q-Q_{\text{true}}}{Q}\right|
$$

is of the order of the machine epsilon $\varepsilon_\text{mach}$, i.e. $\Delta < C\varepsilon_{\text{mach}}$ where $C$ is a small integer number. As we have seen above, $\Delta$ can be much larger than the machine epsilon if $Q$ is computed with numbers of vastly differing size.

## Guidelines for working with numbers in Python
The examples above should have convinced you that some care has to be taken when working with numbers in Python. For integers, watch out for overflow errors. For real numbers there are some guidelines to follow:

* Always try to work with numbers that are of comparable size, ideally $\approx 1$.
* Remember that due to rounding in floating point arithmetic, calculations have a relative error which is a small multiple of $\varepsilon_{\text{mach}}$. For double precision numbers this typically means that the relative error is of the order $\approx 10^{-16}-10^{-15}$. In other words, only the first 14 digits of the result are correct.
* **Never** compare floating point numbers with the `==` operator. Use the `abs()` function for computing the absoluting value of the difference instead:
```Python
a = 4/3-1
a == 1/3              # not correct
abs(a - 1/3) < 1.E-14 # correct
```

Here is a another example which uses the numpy square root function (`np.sqrt()`) to illustrates the final bulletpoint. 

In [ ]:
import numpy as np
a = np.sqrt(2.)
if a**2 == 2: # not correct
    print ("a^2 == 2 is true")
else:
    print ("a^2 == 2 is false")
if abs(a**2 - 2) < 1.E-14:
    print ("|a^2 - 2| < 1.E-14 is true")
else:
    print ("|a^2 - 2| < 1.E-14 is false")


## Check your understanding
Solutions to the following exercises can be found at the very end of this notebook.

**Q1**: Which *unsigned* integer number is represented by the following binary representation:

$$
10110011_2
$$



**Q2**: Which *signed* integer number is represented by the following binary representation:

$$
10110011_2^-
$$

**Q3**: What would be the largest *signed* number that could be represented with 256 bits?

1. $~~~~2^{256}$
2. $~~~~2^{255}-1$
3. $~~~~2^{255}$
4. $~~~~2^{128}$

**Q4**: Single precision floating point numbers use 23 bits to represent the mantissa. What is the largest positive single precision number $x_{\text{tiny}}^{\text{(sp)}}$ such that Python can not distinguish $1+x_{\text{tiny}}^{\text{(sp)}}$? from $1$?

1. $~~~~x_{\text{tiny}}^{\text{(sp)}}\approx 5.96\cdot 10^{-8}$
2. $~~~~x_{\text{tiny}}^{\text{(sp)}}\approx 1.19\cdot 10^{-7}$
3. $~~~~x_{\text{tiny}}^{\text{(sp)}} = 10^{-23}$

## Proof of signed integer representation with Two's Complement (not examinable)
To prove that

$$
(\boldsymbol{1}b_{N-2}b_{N-3}\dots b_2b_1b_0)_2^- =
  -1-\sum_{j=0}^{N-2}\overline{b}_j2^j = -1 - (\overline{b}_{N-2}\overline{b}_{N-3}\dots \overline{b}_2\overline{b}_1\overline{b}_0)_2
$$

we use that $\overline{b}_j=1-b_j$ and the identity

$$\sum_{j=0}^{N-2} 2^j = 2^{N-1}-1$$

With this and the definition of the signed binary representation we find

$$
\begin{aligned}
  (\boldsymbol{1}b_{N-2}b_{N-3}\dots b_2b_1b_0)_2^- &= -2^{N-1} + \sum_{j=0}^{N-2} b_j 2^{j}\\
&= -1 - \sum_{j=0}^{N-2} 2^j +  \sum_{j=0}^{N-2} b_j2^j\qquad\text{use $2^{N-1} = \sum_{j=0}^{N-1}2^j+1$}\\
&= -1 - \sum_{j=0}^{N-2} (1-b_j)2^{j}\qquad{\text{combine sums}}\\
&= -1 - \sum_{j=0}^{N-2} \overline{b}_j2^{j} = -1 - (\overline{b}_{N-2}\overline{b}_{N-3}\dots \overline{b}_2\overline{b}_1\overline{b}_0)_2
\end{aligned}
$$

## Answers to "Check your Understanding" questions
**Q1**: 179, **Q2**: -77, **Q3**: answer #2, **Q4**: answer #1